# Visualizing Neuropixel Probe Locations
It can be handy to know the location and trajectory of the probes that obtain the data. Some NWB files have an **electrodes** field which store these locations. To be more precise, they contain arrays of **CCF** coordinates. [CCF](https://community.brain-map.org/t/allen-mouse-ccf-accessing-and-using-related-data-and-tools/359) is a framework which allows us to represent locations in the brain with coordinates that are relative to brain structure. This notebook uses CCF coordinate data in an extracellular electrophysiology NWB file to render the locations of the [Neuropixel](https://www.neuropixels.org/probe) probes that were used.

To be able to render locations in a brain, you don't need to know how CCF works except that it is a system of coordinates. You also need to ensure that the brain model being used to visualize probes are appropriate for the CCF data used. In this case, we use a mouse brain atlas to work with our mouse CCF data.

We also use **[brainrender](https://github.com/brainglobe/brainrender)** as the engine by which we 3D render the brain and the probe coordinates.

### Environment Setup

In [1]:
from dandi import dandiapi
from pynwb import NWBHDF5IO
from brainrender import Scene
from brainrender.actors import Points
from brainrender._colors import get_random_colors
import numpy as np
### sets up viewing window
from vedo import embedWindow
embedWindow(None)

### Downloading an NWB File
If you don't already have a file to analyze, you can use a file from The Allen Institute's `Visual Coding - Neuropixels` dataset. If you want to choose your own file to download, set `dandiset_id` and `dandi_filepath` accordingly.

In [38]:
dandiset_id = "000021"
dandi_filepath = "sub-703279277/sub-703279277_ses-719161530_probe-729445648_ecephys.nwb"
download_loc = "~/data"

In [39]:
my_dandiset = dandiapi.DandiAPIClient().get_dandiset(dandiset_id)
file = my_dandiset.get_asset_by_path(dandi_filepath)
filename = filepath.split("/")[-1]
# this may take awhile, especially if the file to download is large
file.download(f"{download_loc}/{filename}")

print(f"Downloaded file to {download_loc}/{filename}")

Downloaded file to ../../../data/sub-699733573_ses-715093703.nwb


### Extracting NWB CCF Coordinates
Here, you can read the NWB file you're interested in viewing. Specify your file of interest's relative file path in `nwb_filepath`. From there, the file will be read and the probe unit coordinates will be extracted and turned into a numpy array.

Note that this will only work with ecephys NWB files which have a valid **electrodes** field.

In [40]:
nwb_filepath = f"{download_loc}/{filename}"

In [41]:
### read the nwb file
io = NWBHDF5IO(nwb_filepath, mode="r", load_namespaces=True)
nwb = io.read()

Exception ignored in: <function Scene.__del__ at 0x0000016A801235E0>
Traceback (most recent call last):
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\brainrender\scene.py", line 102, in __del__
    self.close()
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\brainrender\render.py", line 281, in close
    self.plotter.close()
AttributeError: 'NoneType' object has no attribute 'close'


In [42]:
### read the x,y,z ccf coordinates and generate points
xs = nwb.electrodes.x
ys = nwb.electrodes.y
zs = nwb.electrodes.z
n = min(len(xs), len(ys), len(zs))
points = np.array([[xs[i], ys[i], zs[i]] for i in range(n)])

### Creating a Scene
The scene-to-be-rendered is instantiated from a brain atlas within BrainGlobe's [Atlas API](https://gin.g-node.org/brainglobe/atlases). This is used to select a brain model. In this case, we use The 100 micrometer Allen Mouse Brain Atlas, but you could choose others to suit your purposes. It may take awhile to download, but you should only have to do it once. From there, you can select any number of brain regions to highlight in your render and how to color them. Below, VISp is chosen to highlight green. You can choose any number of brain regions and how to color them.

In [43]:
### Below, VISp is selected for coloring. Multiple region arguments can be supplied if needed.
### Region names and IDs can be viewed with each of these lines
# print(scene.atlas.lookup_df.head())
# print(scene.atlas.lookup_df)
# print(scene.atlas.hierarchy)

In [44]:
scene = Scene(atlas_name="allen_mouse_100um", title='Probes')
scene.add_brain_region("VISp", alpha=0.15, color="green")

╭──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                      │
│   brainrender.Actor:                                                                 │
│   ──────────────────────                                                             │
│                                                                                      │
│   name: VISp                                                                         │
│   type: brain region                                                                 │
│   ────────────────────────────────────────────────────────────────────────────────   │
│   center of mass: [9121 1260 5672]                                                   │
│   number of vertices: 3532                                                           │
│   dimensions: [ 7695 10361   370  2248  2079  9300]                                  │
│   color: [0.         0.50196078 0.        ]                                          │
│                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [45]:
### add the points to the scene all as one color
# scene.add(Points(points, name="units", colors="red"))

### add the points to the scene, coloring them based on CCF location label
for location in set(nwb.electrodes.location):
    these_points = np.array([points[i] for i in range(n) if nwb.electrodes.location[i] == location])
    scene.add(Points(these_points, colors=get_random_colors()))

### Rendering
Rendering is as simple as `scene.render()`.
This will create a pop-up window with the interactive 3D rendering of your scene. 
When you're done with it, press "Esc" to close the window!

In [46]:
scene.render()